In [1]:
# import necessary packages
import time
import datetime

from collections import Counter

import math
import numpy as np
import pandas as pd
import pickle

from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

import matplotlib
from matplotlib.transforms import Bbox
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

import astropy.table as at
from astropy.table import MaskedColumn
from astropy.io import fits
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord
from astropy.coordinates import Angle
import astropy.units as u
from astropy.visualization import PercentileInterval, AsinhStretch
from astroquery.mast import Catalogs
from astroquery.sdss import SDSS
from astroquery.simbad import Simbad

import light_curve as lc
from itertools import chain

import os
import sys
import shutil
import glob
import json

import warnings
warnings.filterwarnings("ignore")

plt.style.use('fig_publication.mplstyle')

In [2]:
# Functions to extract light-curve features
def replace_magn_with_flux(s):
    if 'magnitude' in s:
        return s.replace('magnitudes', 'fluxes').replace('magnitude', 'flux')
    return f'{s} for flux light curve'

def create_base_features_class(
        magn_extractor,
        flux_extractor,
        bands=('R', 'g',),
    ):
    feature_names = ([f'{name}_magn' for name in magn_extractor.names]
                     + [f'{name}_flux' for name in flux_extractor.names])
    
    property_names = {band: [f'feature_{name}_{band}'.lower()
                             for name in feature_names]
                      for band in bands}
    
    features_count = len(feature_names)
    
    return feature_names, property_names, features_count


MAGN_EXTRACTOR = lc.Extractor(
    lc.Amplitude(),
    lc.AndersonDarlingNormal(),
    lc.BeyondNStd(1.0),
    lc.BeyondNStd(2.0),
    lc.Cusum(),
    lc.EtaE(),
    lc.InterPercentileRange(0.02),
    lc.InterPercentileRange(0.1),
    lc.InterPercentileRange(0.25),
    lc.Kurtosis(),
    lc.LinearFit(),
    lc.LinearTrend(),
    lc.MagnitudePercentageRatio(0.4, 0.05),
    lc.MagnitudePercentageRatio(0.2, 0.05),
    lc.MaximumSlope(),
    lc.Mean(),
    lc.MedianAbsoluteDeviation(),
    lc.PercentAmplitude(),
    lc.PercentDifferenceMagnitudePercentile(0.05),
    lc.PercentDifferenceMagnitudePercentile(0.1),
    lc.MedianBufferRangePercentage(0.1),
    lc.MedianBufferRangePercentage(0.2),
    lc.Periodogram(
        peaks=5,
        resolution=10.0,
        max_freq_factor=2.0,
        nyquist='average',
        fast=True,
        features=(
            lc.Amplitude(),
            lc.BeyondNStd(2.0),
            lc.BeyondNStd(3.0),
            lc.StandardDeviation(),
        ),
    ),
    lc.ReducedChi2(),
    lc.Skew(),
    lc.StandardDeviation(),
    lc.StetsonK(),
    lc.WeightedMean(),
)

FLUX_EXTRACTOR = lc.Extractor(
    lc.AndersonDarlingNormal(),
    lc.Cusum(),
    lc.EtaE(),
    lc.ExcessVariance(),
    lc.Kurtosis(),
    lc.MeanVariance(),
    lc.ReducedChi2(),
    lc.Skew(),
    lc.StetsonK(),
)

def remove_simultaneous_alerts(table):
    """Remove alert duplicates"""
    dt = np.diff(table['MJD'], append=np.inf)
    return table[dt != 0]
    
def get_detections(photometry, band):
    """Extract clean light curve in given band from locus photometry"""
    band_lc = photometry[(photometry['Filter'] == band) & (~photometry['MAG'].isna())]
    idx = ~MaskedColumn(band_lc['MAG']).mask
    detections = remove_simultaneous_alerts(band_lc[idx])
    return detections

In [3]:
def extract_lc_features(k2, lightcurve):
    

    min_obs_count=4 # 4 is what model is trained on
    lightcurve = lightcurve
    
    feature_names, property_names, features_count = create_base_features_class(MAGN_EXTRACTOR, FLUX_EXTRACTOR)

    g_obs = list(get_detections(lightcurve, 'g').MJD.values)
    r_obs = list(get_detections(lightcurve, 'R').MJD.values)
    mjd_l = sorted(g_obs+r_obs)

    lc_properties_d_l = []
    len_det_counter_r,len_det_counter_g = 0,0

    band_lc = lightcurve[(~lightcurve['MAG'].isna())]
    idx = ~MaskedColumn(band_lc['MAG']).mask
    all_detections = remove_simultaneous_alerts(band_lc[idx])
    for ob, mjd in enumerate(mjd_l): # requires 4 obs
        # do time evolution of detections - in chunks

        detections_pb = all_detections[all_detections['MJD'].values <= mjd]
        #print(detections)
        lc_properties_d={}
        for band, names in property_names.items():
            detections = detections_pb[detections_pb['Filter'] == band]

            # Ensure locus has >3 obs for calculation
            if (len(detections) < min_obs_count):
                continue
            #print(detections)

            t = detections['MJD'].values
            m = detections['MAG'].values
            merr = detections['MAGERR'].values
            flux = np.power(10.0, -0.4 * m)
            fluxerr = 0.5 * flux * (np.power(10.0, 0.4 * merr) - np.power(10.0, -0.4 * merr))

            magn_features = MAGN_EXTRACTOR(
                t,
                m,
                merr,
                fill_value=None,
            )
            flux_features = FLUX_EXTRACTOR(
                t,
                flux,
                fluxerr,
                fill_value=None,
            )

            # After successfully calculating features, set locus properties and tag
            lc_properties_d["obs_num"] = int(ob)
            lc_properties_d["mjd_cutoff"] = mjd
            lc_properties_d["k2"] = k2
            #print(band, m)
            for name, value in zip(names, chain(magn_features, flux_features)):
                lc_properties_d[name] = value
                #if name == "feature_amplitude_magn_g": print(m, value, band)
            #print("%%%%%%%%")
        lc_properties_d_l.append(lc_properties_d)

    lc_properties_d_l = [d for d in lc_properties_d_l if d]
    lc_properties_df = pd.DataFrame(lc_properties_d_l)
    if len(lc_properties_df) == 0: 
        #print(f"Not enough obs for {k2}. pass!\n")
        return
    #print(f"Extracted LC features for {k2}!")
    
    # Drop NaNs
    lc_properties_df = lc_properties_df[~lc_properties_df.isnull().any(axis=1)].reset_index(drop=True)
    
    return lc_properties_df

In [4]:
# From 106 available features from Kostya's lc_feature_extractor, use the 82 from SNAD Miner paper 
# R and g bands
feature_names_r_g = ['feature_amplitude_magn_r', 
                 'feature_anderson_darling_normal_magn_r',
                 'feature_beyond_1_std_magn_r', 
                 'feature_beyond_2_std_magn_r',
                 'feature_cusum_magn_r', 
                 #'feature_eta_e_magn_r',
                 'feature_inter_percentile_range_2_magn_r',
                 'feature_inter_percentile_range_10_magn_r',
                 'feature_inter_percentile_range_25_magn_r', 
                 'feature_kurtosis_magn_r',
                 'feature_linear_fit_slope_magn_r',
                 'feature_linear_fit_slope_sigma_magn_r',
                 #'feature_linear_fit_reduced_chi2_magn_r', 
                 #'feature_linear_trend_magn_r', # cadence removal
                 #'feature_linear_trend_sigma_magn_r',  # cadence removal
                 'feature_magnitude_percentage_ratio_40_5_magn_r',
                 'feature_magnitude_percentage_ratio_20_5_magn_r',
                 #'feature_maximum_slope_magn_r',
                 'feature_mean_magn_r',
                 'feature_median_absolute_deviation_magn_r',
                 'feature_percent_amplitude_magn_r',
                 'feature_median_buffer_range_percentage_10_magn_r',
                 'feature_median_buffer_range_percentage_20_magn_r',
                 'feature_percent_difference_magnitude_percentile_5_magn_r',
                 'feature_percent_difference_magnitude_percentile_10_magn_r',
                 #'feature_period_0_magn_r',  # should be negated
                 #'feature_period_s_to_n_0_magn_r', # cadence removal
                 #'feature_period_1_magn_r', 
                 #'feature_period_s_to_n_1_magn_r', # cadence removal
                 #'feature_period_2_magn_r', 
                 #'feature_period_s_to_n_2_magn_r', # cadence removal
                 #'feature_period_3_magn_r', 
                 #'feature_period_s_to_n_3_magn_r', # cadence removal
                 #'feature_period_4_magn_r', 
                 #'feature_period_s_to_n_4_magn_r', # cadence removal
                 #'feature_periodogram_amplitude_magn_r',
                 #'feature_periodogram_beyond_2_std_magn_r',  # cadence removal
                 #'feature_periodogram_beyond_3_std_magn_r',  # cadence removal
                 #'feature_periodogram_standard_deviation_magn_r',   # cadence removal
                 #'feature_chi2_magn_r',
                 'feature_skew_magn_r', 
                 'feature_standard_deviation_magn_r',
                 'feature_stetson_k_magn_r',
                 'feature_weighted_mean_magn_r',
                 'feature_anderson_darling_normal_flux_r', 
                 'feature_cusum_flux_r',
                 #'feature_eta_e_flux_r', 
                 'feature_excess_variance_flux_r',
                 'feature_kurtosis_flux_r', 
                 'feature_mean_variance_flux_r',
                 #'feature_chi2_flux_r', 
                 'feature_skew_flux_r',
                 'feature_stetson_k_flux_r',

                 'feature_amplitude_magn_g', 
                 'feature_anderson_darling_normal_magn_g',
                 'feature_beyond_1_std_magn_g', 
                 'feature_beyond_2_std_magn_g',
                 'feature_cusum_magn_g', 
                 #'feature_eta_e_magn_g',
                 'feature_inter_percentile_range_2_magn_g',
                 'feature_inter_percentile_range_10_magn_g',
                 'feature_inter_percentile_range_25_magn_g', 
                 'feature_kurtosis_magn_g',
                 'feature_linear_fit_slope_magn_g',
                 'feature_linear_fit_slope_sigma_magn_g',
                 #'feature_linear_fit_reduced_chi2_magn_g', 
                 #'feature_linear_trend_magn_g', # cadence removal
                 #'feature_linear_trend_sigma_magn_g',  # cadence removal
                 'feature_magnitude_percentage_ratio_40_5_magn_g',
                 'feature_magnitude_percentage_ratio_20_5_magn_g',
                 #'feature_maximum_slope_magn_g', 
                 'feature_mean_magn_g',
                 'feature_median_absolute_deviation_magn_g',
                 'feature_median_buffer_range_percentage_10_magn_g',
                 'feature_median_buffer_range_percentage_20_magn_g',
                 'feature_percent_amplitude_magn_g',
                 'feature_percent_difference_magnitude_percentile_5_magn_g',
                 'feature_percent_difference_magnitude_percentile_10_magn_g',
                 #'feature_period_0_magn_g',  # should be negated
                 #'feature_period_s_to_n_0_magn_g', # cadence removal
                 #'feature_period_1_magn_g', 
                 #'feature_period_s_to_n_1_magn_g', # cadence removal
                 #'feature_period_2_magn_g', 
                 #'feature_period_s_to_n_2_magn_g', # cadence removal
                 #'feature_period_3_magn_g', 
                 #'feature_period_s_to_n_3_magn_g', # cadence removal
                 #'feature_period_4_magn_g', 
                 #'feature_period_s_to_n_4_magn_g', # cadence removal
                 #'feature_periodogram_amplitude_magn_g',
                 #'feature_periodogram_beyond_2_std_magn_g',  # cadence removal
                 #'feature_periodogram_beyond_3_std_magn_g', # cadence removal
                 #'feature_periodogram_standard_deviation_magn_g',  # cadence removal
                 #'feature_chi2_magn_g',
                 'feature_skew_magn_g', 
                 'feature_standard_deviation_magn_g',
                 'feature_stetson_k_magn_g', 
                 'feature_weighted_mean_magn_g',
                 'feature_anderson_darling_normal_flux_g', 
                 'feature_cusum_flux_g',
                 #'feature_eta_e_flux_g', 
                 'feature_excess_variance_flux_g',
                 'feature_kurtosis_flux_g', 
                 'feature_mean_variance_flux_g',
                 #'feature_chi2_flux_g', 
                 'feature_skew_flux_g',
                 'feature_stetson_k_flux_g']   

feature_names_hostgal = [
    #  'Unnamed: 0',
    #  'level_0',
    #  'index',
    #  'objName',
    #  'objAltName1',
    #  'objAltName2',
    #  'objAltName3',
    #  'objID',
    #  'uniquePspsOBid',
    #  'ippObjID',
    #  'surveyID',
    #  'htmID',
    #  'zoneID',
    #  'tessID',
    #  'projectionID',
    #  'skyCellID',
    #  'randomID',
    #  'batchID',
    #  'dvoRegionID',
    #  'processingVersion',
    #  'objInfoFlag',
    #  'qualityFlag',
    #  'raStack',
    #  'decStack',
    #  'raStackErr',
    #  'decStackErr',
    #  'raMean',
    #  'decMean',
    #  'raMeanErr',
    #  'decMeanErr',
    #  'epochMean',
    #  'posMeanChisq',
    #  'cx',
    #  'cy',
    #  'cz',
    #  'lambda',
    #  'beta',
    #  'l',
    #  'b',
    #  'nStackObjectRows',
    #  'nStackDetections',
    #  'nDetections',
    #  'ng',
    #  'nr',
    #  'ni',
    #  'nz',
    #  'ny',
    #  'uniquePspsSTid',
    #  'primaryDetection',
    #  'bestDetection',
    #  'gippDetectID',
    #  'gstackDetectID',
    #  'gstackImageID',
    #  'gra',
    #  'gdec',
    #  'graErr',
    #  'gdecErr',
    #  'gEpoch',
    #  'gPSFMag',
    #  'gPSFMagErr',
    #  'gApMag',
    #  'gApMagErr',
    #  'gKronMag',
    #  'gKronMagErr',
    #  'ginfoFlag',
    #  'ginfoFlag2',
    #  'ginfoFlag3',
    #  'gnFrames',
    #  'gxPos',
    #  'gyPos',
    #  'gxPosErr',
    #  'gyPosErr',
    #  'gpsfMajorFWHM',
    #  'gpsfMinorFWHM',
    #  'gpsfTheta',
    #  'gpsfCore',
    #  'gpsfLikelihood',
    #  'gpsfQf',
    #  'gpsfQfPerfect',
    #  'gpsfChiSq',
     'gmomentXX',
     'gmomentXY',
     'gmomentYY',
     'gmomentR1',
     'gmomentRH',
     'gPSFFlux',
    #  'gPSFFluxErr',
     'gApFlux',
    #  'gApFluxErr',
    #  'gApFillFac',
    #  'gApRadius',
     'gKronFlux',
    #  'gKronFluxErr',
     'gKronRad',
    #  'gexpTime',
     'gExtNSigma',
    #  'gsky',
    #  'gskyErr',
    #  'gzp',
    #  'gPlateScale',
    #  'rippDetectID',
    #  'rstackDetectID',
    #  'rstackImageID',
    #  'rra',
    #  'rdec',
    #  'rraErr',
    #  'rdecErr',
    #  'rEpoch',
    # 'rPSFMag',
    #  'rPSFMagErr',
    # 'rApMag',
    #  'rApMagErr',
    # 'rKronMag',
    #  'rKronMagErr',
    #  'rinfoFlag',
    #  'rinfoFlag2',
    #  'rinfoFlag3',
    #  'rnFrames',
    #  'rxPos',
    #  'ryPos',
    #  'rxPosErr',
    #  'ryPosErr',
    #  'rpsfMajorFWHM',
    #  'rpsfMinorFWHM',
    #  'rpsfTheta',
    #  'rpsfCore',
    #  'rpsfLikelihood',
    #  'rpsfQf',
    #  'rpsfQfPerfect',
    #  'rpsfChiSq',
     'rmomentXX',
     'rmomentXY',
     'rmomentYY',
     'rmomentR1',
     'rmomentRH',
    'rPSFFlux',
    #  'rPSFFluxErr',
    'rApFlux',
    #  'rApFluxErr',
    #  'rApFillFac',
    # 'rApRadius',
    'rKronFlux',
    #  'rKronFluxErr',
    'rKronRad',
    #  'rexpTime',
     'rExtNSigma',
    #  'rsky',
    #  'rskyErr',
    #  'rzp',
    #  'rPlateScale',
    #  'iippDetectID',
    #  'istackDetectID',
    #  'istackImageID',
    #  'ira',
    #  'idec',
    #  'iraErr',
    #  'idecErr',
    #  'iEpoch',
    #  'iPSFMag',
    #  'iPSFMagErr',
    #  'iApMag',
    #  'iApMagErr',
    #  'iKronMag',
    #  'iKronMagErr',
    #  'iinfoFlag',
    #  'iinfoFlag2',
    #  'iinfoFlag3',
    #  'inFrames',
    #  'ixPos',
    #  'iyPos',
    #  'ixPosErr',
    #  'iyPosErr',
    #  'ipsfMajorFWHM',
    #  'ipsfMinorFWHM',
    #  'ipsfTheta',
    #  'ipsfCore',
    #  'ipsfLikelihood',
    #  'ipsfQf',
    #  'ipsfQfPerfect',
    #  'ipsfChiSq',
      'imomentXX',
     'imomentXY',
     'imomentYY',
     'imomentR1',
     'imomentRH',
     'iPSFFlux',
    #  'iPSFFluxErr',
     'iApFlux',
    #  'iApFluxErr',
    #  'iApFillFac',
    #  'iApRadius',
     'iKronFlux',
    #  'iKronFluxErr',
     'iKronRad',
    #  'iexpTime',
      'iExtNSigma',
    #  'isky',
    #  'iskyErr',
    #  'izp',
    #  'iPlateScale',
    #  'zippDetectID',
    #  'zstackDetectID',
    #  'zstackImageID',
    #  'zra',
    #  'zdec',
    #  'zraErr',
    #  'zdecErr',
    #  'zEpoch',
    #  'zPSFMag',
    #  'zPSFMagErr',
    #  'zApMag',
    #  'zApMagErr',
    #  'zKronMag',
    #  'zKronMagErr',
    #  'zinfoFlag',
    #  'zinfoFlag2',
    #  'zinfoFlag3',
    #  'znFrames',
    #  'zxPos',
    #  'zyPos',
    #  'zxPosErr',
    #  'zyPosErr',
    #  'zpsfMajorFWHM',
    #  'zpsfMinorFWHM',
    #  'zpsfTheta',
    #  'zpsfCore',
    #  'zpsfLikelihood',
    #  'zpsfQf',
    #  'zpsfQfPerfect',
    #  'zpsfChiSq',
      'zmomentXX',
     'zmomentXY',
     'zmomentYY',
     'zmomentR1',
     'zmomentRH',
     'zPSFFlux',
    # #  'zPSFFluxErr',
     'zApFlux',
    # #  'zApFluxErr',
    # #  'zApFillFac',
    # #  'zApRadius',
     'zKronFlux',
    # #  'zKronFluxErr',
     'zKronRad',
    # #  'zexpTime',
      'zExtNSigma',
    #  'zsky',
    #  'zskyErr',
    #  'zzp',
    #  'zPlateScale',
    #  'yippDetectID',
    #  'ystackDetectID',
    #  'ystackImageID',
    #  'yra',
    #  'ydec',
    #  'yraErr',
    #  'ydecErr',
    #  'yEpoch',
    #  'yPSFMag',
    #  'yPSFMagErr',
    #  'yApMag',
    #  'yApMagErr',
    #  'yKronMag',
    #  'yKronMagErr',
    #  'yinfoFlag',
    #  'yinfoFlag2',
    #  'yinfoFlag3',
    #  'ynFrames',
    #  'yxPos',
    #  'yyPos',
    #  'yxPosErr',
    #  'yyPosErr',
    #  'ypsfMajorFWHM',
    #  'ypsfMinorFWHM',
    #  'ypsfTheta',
    #  'ypsfCore',
    #  'ypsfLikelihood',
    #  'ypsfQf',
    #  'ypsfQfPerfect',
    #  'ypsfChiSq',
      'ymomentXX',
      'ymomentXY',
      'ymomentYY',
      'ymomentR1',
      'ymomentRH',
      'yPSFFlux',
    # #   'yPSFFluxErr',
      'yApFlux',
    # #   'yApFluxErr',
    # #   'yApFillFac',
    # #  'yApRadius',
     'yKronFlux',
    # #  'yKronFluxErr',
     'yKronRad',
    # #  'yexpTime',
      'yExtNSigma',
    #  'ysky',
    #  'yskyErr',
    #  'yzp',
    #  'yPlateScale',
    #  'distance',
    #  'SkyMapper_StarClass',
    #  'gelong',
    #  'g_a',
    #  'g_b',
    #  'g_pa',
    #  'relong',
    #  'r_a',
    #  'r_b',
    #  'r_pa',
    #  'ielong',
    #  'i_a',
    #  'i_b',
    #  'i_pa',
    #  'zelong',
    #  'z_a',
    #  'z_b',
    #  'z_pa',
       'i-z', # try throwing in
    #    'g-r',
    #    'r-i',
    #    'g-i',
    #    'z-y',
    #   'g-rErr',
    #   'r-iErr',
    #   'i-zErr',
    #   'z-yErr',
     'gApMag_gKronMag',
     'rApMag_rKronMag',
     'iApMag_iKronMag',
     'zApMag_zKronMag',
     'yApMag_yKronMag',
     '7DCD', 
    #  'NED_name',
    #  'NED_type',
    #  'NED_vel',
    #  'NED_redshift',
    #  'NED_mag',
    #  'class',
       'dist/DLR',
    #   'dist',
    #  'TransientClass',
    #  'TransientRA',
    #  'TransientDEC'
       ]


lc_and_host_features = feature_names_r_g + feature_names_hostgal

In [5]:
# Hyperparameters for best AD model
n_estimators=100 
max_depth=35 
random_state=11
max_features=35 # {“sqrt”, “log2”, None}, int or float, default=”sqrt” - sqrt(120) ~ 10
class_weight={"Normal": 1, "Other": 1} #"balanced"

model_path = f"/Users/patrickaleo/Desktop/Illinois/LAISS-antares/repo/RFC/SMOTE_train_test_70-30_min14_kneighbors8/cls=binary_n_estimators={n_estimators}_max_depth={max_depth}_rs={random_state}_max_feats={max_features}_cw=balanced/model"
    
with open(f'{model_path}/cls=binary_n_estimators={n_estimators}_max_depth={max_depth}_rs={random_state}_max_feats={max_features}_cw=balanced.pkl', 'rb') as f:
    clf = pickle.load(f)


def plot_RFC_prob_vs_lc_ztfid(clf, k2, cls, photoz, anom_thresh, lc_and_hosts_df, lc_and_hosts_df_120d, lightcurve, savefig, figure_path, plot=False):
    anom_thresh = anom_thresh
    anom_obj_df = lc_and_hosts_df_120d

    try:
        pred_prob_anom = 100 * clf.predict_proba(anom_obj_df)
        pred_prob_anom[:, 0] = [round(a, 1) for a in pred_prob_anom[:, 0]]
        pred_prob_anom[:, 1] = [round(b, 1) for b in pred_prob_anom[:, 1]]
        num_anom_epochs = len(np.where(pred_prob_anom[:, 1]>=anom_thresh)[0])
    except:
        #print(f"{k2} has some NaN host galaxy values from PS1 catalog. Skip!")
        return

    try: 
        anom_idx = lc_and_hosts_df.iloc[np.where(pred_prob_anom[:, 1]>=anom_thresh)[0][0]].obs_num
        anom_idx_is = True
        #print("Anomalous during timeseries!")

    except: 
        #print(f"Prediction doesn't exceed anom_threshold of {anom_thresh}% for {k2}.")
        anom_idx_is = False

    max_anom_score = max(pred_prob_anom[:, 1])
    #print("max_anom_score", round(max_anom_score, 1))
    #print("num_anom_epochs", num_anom_epochs)
    
    if plot:
        df_ref = lightcurve

        df_ref_g = df_ref[(df_ref.Filter == 'g') & (~df_ref.MAG.isna())]
        df_ref_r = df_ref[(df_ref.Filter == 'R') & (~df_ref.MAG.isna())]

        mjd_idx_at_min_mag_r_ref = df_ref_r[['MAG']].reset_index().idxmin().MAG
        mjd_idx_at_min_mag_g_ref = df_ref_g[['MAG']].reset_index().idxmin().MAG

        fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize=(7,10))
        ax1.invert_yaxis()
        ax1.errorbar(x=df_ref_r.MJD, y=df_ref_r.MAG, yerr=df_ref_r.MAGERR, fmt='o', c='r', label=r'ZTF-$r$')
        ax1.errorbar(x=df_ref_g.MJD, y=df_ref_g.MAG, yerr=df_ref_g.MAGERR, fmt='o', c='g', label=r'ZTF-$g$')
        if anom_idx_is == True:
            ax1.axvline(x=lc_and_hosts_df[lc_and_hosts_df.obs_num == anom_idx].mjd_cutoff.values[0], 
                        label="Tagged anomalous", color='darkblue', ls='--')
            mjd_cross_thresh = round(lc_and_hosts_df[lc_and_hosts_df.obs_num == anom_idx].mjd_cutoff.values[0], 3)

            left, right = ax1.get_xlim()
            mjd_anom_per = (mjd_cross_thresh - left)/(right - left)
            plt.text(mjd_anom_per+0.073, -0.075, f"t = {int(mjd_cross_thresh)}", horizontalalignment='center',
             verticalalignment='center', transform=ax1.transAxes, fontsize=16)
            print("MJD crossed thresh:", mjd_cross_thresh)

        ax2.plot(lc_and_hosts_df.mjd_cutoff, pred_prob_anom[:, 0], label=r'$p(Normal)$')
        ax2.plot(lc_and_hosts_df.mjd_cutoff, pred_prob_anom[:, 1], label=r'$p(Anomaly)$')

        ax1.set_title(fr"{k2} ({cls}, $photo-z$={photoz})" , pad=25)
        plt.xlabel('MJD')
        ax1.set_ylabel('Magnitude')
        ax2.set_ylabel('Probability (%)')

        if anom_idx_is == True: ax1.legend(loc='upper right', ncol=3, bbox_to_anchor=(1.0,1.12), frameon=False, fontsize=14)
        else: ax1.legend(loc='upper right', ncol=2, bbox_to_anchor=(0.75,1.12), frameon=False, fontsize=14)
        ax2.legend(loc='upper right', ncol=2, bbox_to_anchor=(0.87,1.12), frameon=False, fontsize=14)

        ax1.grid(True)
        ax2.grid(True)

        if savefig:
            plt.savefig(f"{figure_path}/{k2}_{cls}_AD_run_timeseries.pdf", dpi=300, bbox_inches='tight')

        plt.show()
    
    if anom_idx_is == True:
        anom_idx_is = "Anomaly"
    elif anom_idx_is == False:
        anom_idx_is = "Normal"
    else: 
        anom_idx_is = "???"
        
    return k2, anom_idx_is

In [6]:


# Specify the path to your JSON file
json_file_path = 'ZTF_sims_from_Alex/ZTFAnomalies_FORPALEO.json'

# Open the JSON file for reading
with open(json_file_path, 'r', encoding='utf-8') as json_file:
    # Initialize an empty list to store parsed data
    data_list = []

    # Iterate over the lines in the file to read and parse each JSON object
    for line in json_file:
        try:
            # Load and parse the JSON object from the current line
            data = json.loads(line)
            
            # Append the parsed data to the list
            data_list.append(data)
        except json.JSONDecodeError as e:
            print(f"Error parsing JSON: {str(e)}")
            continue

# Now, 'data_list' contains all the parsed data from the JSON file


In [7]:
# Assuming 'data_list' contains your parsed JSON data

# Initialize an empty set to store unique keys
all_keys = set()

# Iterate through the parsed data
for item in data_list:
    # Check if the item is a dictionary (JSON object)
    if isinstance(item, dict):
        # Update the set of keys with the keys from this dictionary
        all_keys.update(item.keys())

# Now, 'all_keys' contains all the unique keys present in your JSON data
print(all_keys)


{'HOSTGAL_MAG_z', 'HOSTGAL_MAG_g', 'HOSTGAL_MAG_i', 'Flux_Err', 'MAGERR', 'HOSTGAL_MAGERR_r', 'SIM_HOSTLIB_GALID', 'MAG', 'HOSTGAL_PHOTOZ', 'HOSTGAL_MAGERR_g', 'HOSTGAL_PHOTOZ_ERR', 'HOSTGAL_MAG_r', 'REDSHIFT_HELIO', 'MJD', 'HOSTGAL_MAGERR_i', 'Flux', 'MJD_TRIGGER', 'MWEBV', 'Filter', 'Type', 'HOSTGAL_MAGERR_z', 'HOSTGAL_DDLR'}


In [8]:
anomaly_distr_d = {'PISN-STELLA_HYDROGENIC': 978,
         'TDE': 985,
         'KN_B19': 2,
         'SNIIn-MOSFIT': 242,
         'SNIa-91bg': 1023,
         'SNIax': 239,
         'SNIIn+HostXT': 1774,
         'SNII': 2330,
         'CART': 90,
         'PISN-STELLA_HECORE': 595,
         'SLSNI': 4166,
         'PISN-MOSFIT': 4152,
         'SNIb': 989,
         'SNIa': 3530,
         'SNIcBL+HostXT_V19': 1145,
         'SNIc': 2532,
         'ILOT': 1}

In [9]:
uniq_cls_l = np.unique(list(data['Type'].values()))
uniq_cls_l

array(['CART', 'ILOT', 'KN_B19', 'PISN-MOSFIT', 'PISN-STELLA_HECORE',
       'PISN-STELLA_HYDROGENIC', 'SLSNI', 'SNII', 'SNIIn+HostXT',
       'SNIIn-MOSFIT', 'SNIa', 'SNIa-91bg', 'SNIax', 'SNIb', 'SNIc',
       'SNIcBL+HostXT_V19', 'TDE'], dtype='<U22')

In [10]:
dataset_bank = pd.read_csv('../loci_dbs/alerce_cut/dataset_bank.csv.gz', compression='gzip', index_col=0)
host_dataset_bank = dataset_bank[feature_names_hostgal]
host_dataset_bank

,gmomentXX,gmomentXY,gmomentYY,gmomentR1,gmomentRH,gPSFFlux,gApFlux,gKronFlux,gKronRad,gExtNSigma,...,yKronRad,yExtNSigma,i-z,gApMag_gKronMag,rApMag_rKronMag,iApMag_iKronMag,zApMag_zKronMag,yApMag_yKronMag,7DCD,dist/DLR
ztf_object_id,,,,,,,,,,,,,,,,,,,,,
ZTF17aaadars,0.182697,0.000057,0.206433,0.899402,0.707021,0.000008,0.000008,0.000008,2.24851,-0.771992,...,2.28946,0.923409,0.257099,-0.120399,-0.177700,-0.116199,-0.128300,-0.208900,48.409333,0.063386
ZTF17aaahrni,0.395019,0.035295,0.428865,1.574710,0.911899,0.000003,0.000008,0.000009,3.93677,8.333380,...,2.43505,6.442500,0.096600,0.164299,-0.071001,0.051300,-0.024000,-0.145601,22.882911,2.639232
ZTF17aaaizmo,0.280209,0.020304,0.289936,3.348710,1.155740,0.000057,0.000141,0.000310,8.37179,15.204500,...,5.85443,28.151199,0.027800,0.855101,0.652599,0.618401,0.684500,0.695499,14.803999,4.788567
ZTF17aabtbti,0.345283,-0.014541,0.300773,1.260110,0.941188,0.000004,0.000007,0.000007,3.15028,4.822180,...,2.25483,3.383140,0.175501,-0.055199,0.066099,-0.006701,-0.024200,-0.064001,5.578753,11.742125
ZTF17aachsxt,0.330886,0.020398,0.289500,1.696890,1.039760,0.000024,0.000046,0.000051,4.24223,18.275499,...,3.51504,20.063499,-0.094900,0.122499,0.208000,0.324200,0.378099,0.192301,45.767392,4.255986
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZTF22aaboeex,0.308624,0.007761,0.309511,3.498180,1.136470,0.000074,0.000165,0.000340,8.74544,24.032301,...,6.76913,26.586201,0.233500,0.784500,0.708401,0.683500,0.644199,0.548500,25.406284,1.134230
ZTF22aaboiqa,0.271177,0.036831,0.286689,1.241780,0.951537,0.000035,0.000047,0.000045,3.10445,7.224710,...,2.79864,5.512640,0.042000,-0.048300,-0.035500,-0.026699,0.067301,-0.125101,10.365118,0.148034
ZTF22aabovlw,0.269753,0.002017,0.252035,2.489740,1.080980,0.000083,0.000170,0.000262,6.22435,26.303900,...,6.67416,21.558701,0.225199,0.468000,0.606100,0.483999,0.585100,0.647001,33.811442,4.407910


In [11]:
median_host_vals_dict = {}

for f in host_dataset_bank.columns:
    median_val = np.median(host_dataset_bank[f])
    print(f, median_val)
    median_host_vals_dict[f] = median_val

gmomentXX 0.3279579877853393
gmomentXY 0.000289833988063
gmomentYY 0.3296020030975342
gmomentR1 2.16513991355896
gmomentRH 1.1088800430297852
gPSFFlux 4.122230166103691e-05
gApFlux 8.937210077419877e-05
gKronFlux 0.0001254280068678
gKronRad 5.4128499031066895
gExtNSigma 20.0137996673584
rmomentXX 0.2744059860706329
rmomentXY 0.000149056999362
rmomentYY 0.2736389935016632
rmomentR1 2.2395999431610107
rmomentRH 1.0743399858474731
rPSFFlux 6.563430360984057e-05
rApFlux 0.0001446600072085
rKronFlux 0.0002102949947584
rKronRad 5.598999977111816
rExtNSigma 19.280000686645508
imomentXX 0.2426429986953735
imomentXY 0.0001063829986378
imomentYY 0.2424930036067962
imomentR1 2.25954008102417
imomentRH 1.0501999855041504
iPSFFlux 8.896220242604613e-05
iApFlux 0.0001902249932754
iKronFlux 0.000290505005978
iKronRad 5.648849964141846
iExtNSigma 16.920400619506836
zmomentXX 0.2212350070476532
zmomentXY 3.861559889628552e-05
zmomentYY 0.2217189967632293
zmomentR1 2.048959970474243
zmomentRH 1.01489996

In [22]:
%%time

uniq_cls_l = ['CART', 'ILOT', 'KN_B19', 'PISN-MOSFIT', 'PISN-STELLA_HECORE',
       'PISN-STELLA_HYDROGENIC', 'SLSNI', 'SNII', 'SNIIn+HostXT',
       'SNIIn-MOSFIT', 'SNIa', 'SNIa-91bg', 'SNIax', 'SNIb', 'SNIc',
       'SNIcBL+HostXT_V19', 'TDE']
uniq_key_l = ['HOSTGAL_MAGERR_g', 'Filter', 'MAGERR', 'HOSTGAL_MAG_i', 'Flux_Err', 
'HOSTGAL_MAG_g', 'MJD', 'MAG', 'HOSTGAL_PHOTOZ', 'MWEBV', 'REDSHIFT_HELIO', 
'HOSTGAL_PHOTOZ_ERR', 'HOSTGAL_MAGERR_z', 'Flux', 'HOSTGAL_DDLR', 'MJD_TRIGGER', 
'Type', 'SIM_HOSTLIB_GALID', 'HOSTGAL_MAG_r', 'HOSTGAL_MAG_z', 'HOSTGAL_MAGERR_i', 'HOSTGAL_MAGERR_r']

master_dict = dict()
num = 0
num_inspect = 100
for cls in uniq_cls_l:
    print("CLS", cls)
    cls_galid_d = dict()
    for k, v in data.items():
        if k == 'Type':
            for k2, v2 in v.items():
                if v2 == cls: #"CART"
                    num += 1
                    #if num > num_inspect: 
                    #    break
                    cls_galid_d[k2] = "Observed" # do True/False if it passes AD model or not
                    uniq_pbs = np.unique(data["Filter"].get(k2))
                    if len(uniq_pbs) <= 1:
                        cls_galid_d[k2] = "FailCuts"
                        master_dict[cls] = cls_galid_d
                        continue

                    # run LC feat extractor
                    lightcurve = pd.DataFrame(zip(data["MJD"].get(k2), data["Filter"].get(k2), data["Flux"].get(k2), 
                                                  data["Flux_Err"].get(k2), data["MAG"].get(k2), data["MAGERR"].get(k2)),
                                    columns=['MJD', 'Filter', 'Flux', 'Flux_Err', 'MAG', 'MAGERR'])

                    try:
                        lc_properties_df = extract_lc_features(k2=k2, lightcurve=lightcurve)
                        lc_and_hosts_df_120d = lc_properties_df[feature_names_r_g]
                    except Exception as e:
                        #print(e)
                        cls_galid_d[k2] = "FailCuts"
                        master_dict[cls] = cls_galid_d


                    # Convert host features into those that the model can run on
                    for h_feat in feature_names_hostgal:
                        #print(h_feat, median_host_vals_dict.get(h_feat))
                        lc_and_hosts_df_120d[h_feat] = median_host_vals_dict.get(h_feat)


                    # run AD model
                    #print(lc_and_hosts_df_120d, lc_and_hosts_df_120d)
                    k2, anom_idx_is = plot_RFC_prob_vs_lc_ztfid(clf=clf, 
                                              k2=k2, 
                                              cls=v2, 
                                              photoz=round(data["HOSTGAL_PHOTOZ"].get(k2), 3), 
                                              anom_thresh=50, 
                                              lc_and_hosts_df=lc_properties_df, 
                                              lc_and_hosts_df_120d=lc_and_hosts_df_120d, 
                                              lightcurve=lightcurve, 
                                              savefig=False, 
                                              figure_path="Abc", 
                                              plot=False)

                    if anom_idx_is == "Normal":
                        cls_galid_d[k2] = "Normal"
                    elif anom_idx_is == "Anomaly":
                        cls_galid_d[k2] = "Anomaly"
                    else:
                        cls_galid_d[k2] = "???"
                        
                    master_dict[cls] = cls_galid_d

#     # if passes, keep Tagged. Else, not Tagged
# cls_l, num_anom_tagged_l, num_anom_generated_l, anom_recovered_percent_l = [],[],[],[]
# print("Tally:", Counter(cls_galid_d.values()))
# num_anom_tagged = (Counter(cls_galid_d.values()).get('Anomaly'))
# num_anom_generated = anomaly_distr_d.get(cls)
# anom_recovered_percent = round((num_anom_tagged/num_anom_generated)*100, 3)
# print(f"Anomaly % recovered: {num_anom_tagged}/{num_anom_generated}% = {anom_recovered_percent}%")
# cls_l.append(cls), num_anom_tagged_l.append(num_anom_tagged)
# num_anom_generated_l.append(num_anom_generated), anom_recovered_percent_l.append(anom_recovered_percent)

CLS CART


[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_job

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_job

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_job

CLS ILOT
CLS KN_B19
CLS PISN-MOSFIT


[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_job

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_job

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.0s finished


KeyboardInterrupt: 

In [23]:
cls_l, num_anom_tagged_l, num_anom_generated_l, anom_recovered_percent_l,  num_failcuts_l = [],[],[],[],[]

for cls in master_dict.keys():
    print(cls)
    print("Tally:", Counter(master_dict.get(cls).values()))
    num_anom_tagged = (Counter(master_dict.get(cls).values()).get('Anomaly')) 
    num_anom_generated = anomaly_distr_d.get(cls)
    try:
        anom_recovered_percent = round((num_anom_tagged/num_anom_generated), 3)*100
    except:
        num_anom_tagged = 0
        anom_recovered_percent = round((num_anom_tagged/num_anom_generated), 3)*100
    print(f"Anomaly % recovered: {num_anom_tagged}/{num_anom_generated}% = {anom_recovered_percent}%")
    num_failcuts = (Counter(master_dict.get(cls).values()).get('FailCuts'))
    
    if num_failcuts is None:
        num_failcuts = 0
    
    cls_l.append(cls), num_anom_tagged_l.append(num_anom_tagged)
    num_anom_generated_l.append(num_anom_generated), anom_recovered_percent_l.append(anom_recovered_percent), num_failcuts_l.append(num_failcuts)

CART
Tally: Counter({'Normal': 63, 'Anomaly': 27})
Anomaly % recovered: 27/90% = 30.0%
ILOT
Tally: Counter({'FailCuts': 1})
Anomaly % recovered: 0/1% = 0.0%
KN_B19
Tally: Counter({'Normal': 2})
Anomaly % recovered: 0/2% = 0.0%
PISN-MOSFIT
Tally: Counter({'Normal': 60, 'Anomaly': 20, 'Observed': 1})
Anomaly % recovered: 20/4152% = 0.5%


In [24]:
anom_recover_df = pd.DataFrame(zip(cls_l, num_anom_tagged_l, num_anom_generated_l, anom_recovered_percent_l, num_failcuts_l),
                              columns=["cls", "num_anom_tagged", "num_anom_generated", "anom_recovered_percent", "num_failcuts"])
anom_recover_df

,cls,num_anom_tagged,num_anom_generated,anom_recovered_percent,num_failcuts
0,CART,27,90,30.0,0
1,ILOT,0,1,0.0,1
2,KN_B19,0,2,0.0,0
3,PISN-MOSFIT,20,4152,0.5,0
